# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import sys
import re
import pickle
import pandas as pd
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [15]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/zmruhi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/zmruhi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/zmruhi/nltk_data...


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterMessages', engine)
X = df['message']
Y = df.iloc[:, 4:]
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    # creating tokens
    words = word_tokenize(re.sub(r"[^a-zA-Z0-9]", ' ', text.lower()))
    # stemming
    stemmed_words = [PorterStemmer().stem(w) for w in words]
    # lemmatizing
    lemmatizer = WordNetLemmatizer()
    lemmed_words = [lemmatizer.lemmatize(w, pos='n').strip() for w in stemmed_words]
    lemmed_words = [lemmatizer.lemmatize(w, pos='v').strip() for w in stemmed_words]

    return lemmed_words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
                        ('vect', CountVectorizer(tokenizer=tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(RandomForestClassifier()))
                        ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 0)
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fdcd2100550>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
preds = pipeline.predict(X_test)

for i in range(36):
    print(Y_test.columns[i], ':')
    print(classification_report(Y_test.iloc[:,i], preds[:,i]))

related :
              precision    recall  f1-score   support

           0       0.74      0.27      0.39      1242
           1       0.81      0.97      0.88      3957
           2       0.46      0.46      0.46        37

    accuracy                           0.80      5236
   macro avg       0.67      0.56      0.58      5236
weighted avg       0.79      0.80      0.76      5236

request :
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4374
           1       0.88      0.42      0.57       862

    accuracy                           0.90      5236
   macro avg       0.89      0.71      0.76      5236
weighted avg       0.89      0.90      0.88      5236

offer :
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5206
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5236
   macro avg       0.50      0.50      0.50   

/Users/zmruhi/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zmruhi/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zmruhi/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

### 6. Improve your model
Use grid search to find better parameters. 

In [32]:
parameters = {'clf__estimator__n_estimators': [20, 50],
                  'vect__max_df': (0.5, 0.75, 1.0),
                  'clf__estimator__min_samples_split': [2, 3, 5],
                 }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [33]:
category_names = Y.columns

cv.fit(X_train, Y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fdcd2100550>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 3, 5],
                         'clf__estimator__n_estimators': [20, 50],
                         'vect__max_df': (0.5, 0.75, 1.0)})

In [34]:
Y_pred = cv.predict(X_test)

for i in range(len(category_names)):
    print(Y_test.columns[i], ':')
    print(classification_report(Y_test.iloc[:,i], preds[:,i]))

related :
              precision    recall  f1-score   support

           0       0.74      0.27      0.39      1242
           1       0.81      0.97      0.88      3957
           2       0.46      0.46      0.46        37

    accuracy                           0.80      5236
   macro avg       0.67      0.56      0.58      5236
weighted avg       0.79      0.80      0.76      5236

request :
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      4374
           1       0.88      0.42      0.57       862

    accuracy                           0.90      5236
   macro avg       0.89      0.71      0.76      5236
weighted avg       0.89      0.90      0.88      5236

offer :
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5206
           1       0.00      0.00      0.00        30

    accuracy                           0.99      5236
   macro avg       0.50      0.50      0.50   

/Users/zmruhi/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zmruhi/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zmruhi/opt/anaconda3/envs/env_pytorch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

In [35]:
cv.best_params_

{'clf__estimator__min_samples_split': 3,
 'clf__estimator__n_estimators': 50,
 'vect__max_df': 0.5}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [40]:
cv.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.5,
                                 tokenizer=<function tokenize at 0x7fdcd2100550>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(min_samples_split=3,
                                                                        n_estimators=50)))])

### 9. Export your model as a pickle file

In [39]:
pickle.dump(cv.best_estimator_, open('./models/classifier.pkl', "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.